# Fifth Agent Agent

Introducing a critical agent - the agent that brings it all together.

# Planning Agent

There are a number of frameworks out there that support building Agentic Workflows.

OpenAI has OpenAI Agents SDK, LangChain has LangGraph, and there's Autogen from Microsoft, Crew.ai and many others.  

Each of these are abstractions on top of APIs to LLMs; some are lightweight, others come with significant functionality.

It's also perfectly possible - and sometimes considerably easier - to build an agentic solution by calling LLMs directly.

There's been considerable convergence on LLM APIs, and it's not clear that there's a need to sign up for one of the agent ecosystems for many use cases.

Anthropic has an [insightful post](https://www.anthropic.com/research/building-effective-agents) on building effective Agentic architectures that's well worth a read.

# We are going to use OpenAI Agents SDK for this Agent

## And we're using Tools to give our Agent autonomy

In our case, we're going to create an Agent that uses Tools to make decisions about what to do next.

Let's see how it works

In [ ]:
# imports

import os
import json

from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
# We won't go in depth into OpenAI Agents SDK but it's super easy and convenient!

from agents import Agent, Runner, function_tool
from agents.mcp import MCPServerStdio

In [ ]:
# Initialization

openai_api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-5"

In [ ]:
# Use the Scanner agent from before

from price_agents.scanner_agent import ScannerAgent
scanner = ScannerAgent()

# Our tools

The next 3 cells have 3 **fake** functions that we will allow our LLM to call

In [ ]:
@function_tool
def scan_the_internet_for_bargains() -> str:
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print("Fake function to scan the internet - this returns a hardcoded set of deals")
    results = scanner.test_scan()
    return results.model_dump_json()

In [ ]:
@function_tool
def estimate_true_value(description: str) -> str:
    """
    This tool estimates the true value of a product based on a text description of it

    Args:
        description: a description of the product to be estimated
    """
    print(f"Fake function to estimating true value of {description[:20]}... - this always returns $300")
    result = {"description": description, "estimated_true_value": 300}
    return json.dumps(result)

In [ ]:
@function_tool
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float) -> str:
    """
    This tool notifies the user of a great deal, given a description of it, the price of the deal, and the estimated true value

    Args:
        description: a description of the product with the deal
        deal_price: how much the product is being offered for
        estimated_true_value: an estimate of how much this product is actually worth
        url: the web address of the product
    """
    print(f"Fake function to notify user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return "notification sent ok"

## Telling the LLM about the tools it can use, with JSON

"Tool calling" is giving an LLM the power to run code on your computer!

Sounds a bit spooky?

The way it works is a little more mundane. We give the LLM a description of each Tool and the parameters, and we let it inform us if it wants any tool to be run.

See this conversation I had with ChatGPT and see if it is revealing for you!

https://chatgpt.com/share/6856f9aa-c8b4-8012-abdc-09f5da82aa4e

It's not like OpenAI reaches in and runs a function. In the end, we have an if statement that calls our function if the model requests it.

## OpenAI Agents SDK has made this easy for us

The decorator `function_tools` around each of our functions automatically generates the description we need for the LLM

In [ ]:
notify_user_of_deal

In [ ]:
notify_user_of_deal.params_json_schema

In [ ]:
tools = [scan_the_internet_for_bargains, estimate_true_value, notify_user_of_deal]
tools

## And.. MCP

The Model Context Protocol from Anthropic is causing a lot of excitement.

It gives us a really easy way to integrate new capabilities with our agent, as more tools.

Here we will give our agent powers to write to our local filesystem in a directory "sandbox"

MCP is covered in more detail in other sessions (linked in resources) but the basics:

1. You describe them using parameters
2. You create them with `with MCPServerStdio(params=params) as server`
3. You can call `server.session.list_tools()` or pass in `server` 

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))

# parameters describe an MCP server
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as server:
    file_tools = await server.session.list_tools()

In [ ]:
file_tools.tools

In [ ]:
task = """
You are an Autonomous AI Agent that makes use of tools to carry out your mission.
Your mission is to find great deals on bargain products, and notify the user with a push notification and a written file.
First scan the internet for bargains. Then for each deal, estimate its true value - how much it's actually worth.
Finally, pick the single most compelling deal where the deal price is much lower than the estimated true value, and 
send the user a push notification about that deal, and also write or update a file called sandbox/deals.md with a description in markdown.
You must only notify the user about one deal, and be sure to pick the most compelling deal.
Then just respond OK to indicate success.
"""

### And here's where it comes together - just 2 lines of code

Keep in mind: this will use the Tools we provided it, which are the fake functions above

In [ ]:
async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as server:
    agent = Agent(name="Planner", model=MODEL, tools=tools, mcp_servers=[server])
    result = await Runner.run(agent, task)

print(result.final_output)

## And now - putting all of this into a Planning Agent

But instead of these Fake Functions, this Planning Agent will actually call the other Agents!

In [ ]:
from price_agents.autonomous_planning_agent import AutonomousPlanningAgent

In [ ]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
import chromadb
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [ ]:
agent = AutonomousPlanningAgent(collection)

In [ ]:
result = agent.plan()

### Check out the trace

https://platform.openai.com/traces

# Finally - with a Gradio UI

In [22]:
# Reset memory back to 2 deals discovered in the past

from deal_agent_framework import DealAgentFramework
DealAgentFramework.reset_memory()

In [ ]:
!uv run price_is_right.py

In [ ]:
# Put modal back to sleeping after 2 minutes

import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()
pricer.update_autoscaler(scaledown_window=120)